In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
train_df = pd.read_csv('train.csv')
train_df.head(5)

,Unnamed: 0,user_id,business_id,rating,rounded_business_rating,latitude,longitude,average_user_rating,num_user_reviews,avg_user_compliments,reviews_per_year,city,state,name,attributes,review_count,city_state,user_business_pair,price_range,bike_parking,business_accepts_credit_cards,good_for_kids,has_TV,outdoor_seating,restaurants_delivery,restaurants_good_for_groups,restaurants_reservations,restaurants_take_out,restaurants_counter_service,by_appointment_only,wheelchair_accessible,open_24_hours,dogs_allowed,caters,drive_thru,coat_check,good_for_dancing,restaurants_table_service,happy_hour,business_accepts_bitcoin,garage,street,validated,lot,valet,lunch,dinner,casual,city_state_le,name_le,max_user_likes,total_user_likes,avg_user_likes,max_business_likes,total_business_likes,avg_business_likes,max_ub_pair_likes,total_ub_pair_likes,avg_ub_pair_likes
0,0,pmKnNkGVgzhMkwJaTU5apA,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.74,959,0.045881,121.371359,Las Vegas,NV,Pho Kim Long,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2850,Las Vegas-NV,"('pmKnNkGVgzhMkwJaTU5apA', 'yfxDa8RFOvJPQh0rNt...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,454,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
1,1,cQU8IWVEJYcklBJbW9CzdQ,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.59,289,0.179931,32.120889,Las Vegas,NV,Pho Kim Long,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2850,Las Vegas-NV,"('cQU8IWVEJYcklBJbW9CzdQ', 'yfxDa8RFOvJPQh0rNt...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,454,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
2,2,GmQR50bIII7gDNRYEgfXfA,yfxDa8RFOvJPQh0rNtakHA,5.0,3.5,36.126194,-115.193445,4.07,1261,0.298176,139.813183,Las Vegas,NV,Pho Kim Long,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2850,Las Vegas-NV,"('GmQR50bIII7gDNRYEgfXfA', 'yfxDa8RFOvJPQh0rNt...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,454,94743,0.0,0.0,0.000000,1.0,7.0,0.006416,0.0,0.0,0.0
3,3,m4VAw2u8gHrANfXP7CW2mg,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,4.03,58,0.103448,9.810009,Las Vegas,NV,Pho Kim Long,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2850,Las Vegas-NV,"('m4VAw2u8gHrANfXP7CW2mg', 'yfxDa8RFOvJPQh0rNt...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,454,94743,1.0,5.0,0.046729,1.0,7.0,0.006416,0.0,0.0,0.0
4,4,mvK1pv-x0Z5rP4Hz9cWBWw,yfxDa8RFOvJPQh0rNtakHA,4.0,3.5,36.126194,-115.193445,3.88,1641,6.855576,152.098781,Las Vegas,NV,Pho Kim Long,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2850,Las Vegas-NV,"('mvK1pv-x0Z5rP4Hz9cWBWw', 'yfxDa8RFOvJPQh0rNt...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,454,94743,2.0,2.0,0.181818,1.0,7.0,0.006416,0.0,0.0,0.0


In [4]:
test_df = pd.read_csv('test.csv')
test_df.head(5)

,Unnamed: 0,user_id,business_id,rounded_business_rating,latitude,longitude,average_user_rating,num_user_reviews,avg_user_compliments,reviews_per_year,city,state,name,attributes,review_count,city_state,user_business_pair,price_range,bike_parking,business_accepts_credit_cards,good_for_kids,has_TV,outdoor_seating,restaurants_delivery,restaurants_good_for_groups,restaurants_reservations,restaurants_take_out,restaurants_counter_service,by_appointment_only,wheelchair_accessible,open_24_hours,dogs_allowed,caters,drive_thru,coat_check,good_for_dancing,restaurants_table_service,happy_hour,business_accepts_bitcoin,garage,street,validated,lot,valet,lunch,dinner,casual,city_state_le,name_le,max_user_likes,total_user_likes,avg_user_likes,max_business_likes,total_business_likes,avg_business_likes,max_ub_pair_likes,total_ub_pair_likes,avg_ub_pair_likes
0,0,NCRGj36ihdwX-VUCYEX41g,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.13,317,0.378549,42.742889,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('NCRGj36ihdwX-VUCYEX41g', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0
1,1,mySDXsBhY3u0yI-G2oVDkA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,2.73,239,0.200837,23.469196,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('mySDXsBhY3u0yI-G2oVDkA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0
2,2,wLc6l8v7O9hXjrDi3Q7tyw,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.96,102,0.666667,11.978764,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('wLc6l8v7O9hXjrDi3Q7tyw', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,1.0,1.0,0.004808,2.0,11.0,0.012373,0.0,0.0,0.0
3,3,hnhs-uzv_eoE6Pq8nXyufA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.84,203,0.394089,25.142518,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('hnhs-uzv_eoE6Pq8nXyufA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0
4,4,Gr-MqCunME2K_KmsAwjpTA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.12,336,1.309524,37.922078,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('Gr-MqCunME2K_KmsAwjpTA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,1.0,2.0,0.080000,2.0,11.0,0.012373,0.0,0.0,0.0


In [5]:
test_df_with_labels = pd.read_csv('y_test.csv')
test_df_with_labels

,user_id,business_id,rating
0,m5b84-nPPVNLlDtZyVGZ3w,tWdp26XQEJqJGnRmFVJQxA,4.0
1,m5b84-nPPVNLlDtZyVGZ3w,qy4WKXTBkqD4zX0rohYF8Q,4.0
2,m5b84-nPPVNLlDtZyVGZ3w,478TIlfHXfT3wvww54QsPg,3.0
3,m5b84-nPPVNLlDtZyVGZ3w,SxvKjtbq5qRlFoXrU8Bo9Q,2.0
4,m5b84-nPPVNLlDtZyVGZ3w,ErTB7R4rJNUUV8nsUVG26Q,4.0
...,...,...,...
142039,iCqpb2C3TZrB5gdMASf8LQ,N8D-5-fW7EkBdKjNYGhu0A,5.0
142040,iCqpb2C3TZrB5gdMASf8LQ,dT9316C_MJjATvv7GByROg,4.0
142041,iCqpb2C3TZrB5gdMASf8LQ,SAIrNOB4PtDA4gziNCucwg,4.0
142042,iCqpb2C3TZrB5gdMASf8LQ,K7lWdNUhCbcnEvI0NhGewg,4.0


In [6]:
final_test_df = pd.merge(test_df, test_df_with_labels, on=['user_id', 'business_id'])
final_test_df

,Unnamed: 0,user_id,business_id,rounded_business_rating,latitude,longitude,average_user_rating,num_user_reviews,avg_user_compliments,reviews_per_year,city,state,name,attributes,review_count,city_state,user_business_pair,price_range,bike_parking,business_accepts_credit_cards,good_for_kids,has_TV,outdoor_seating,restaurants_delivery,restaurants_good_for_groups,restaurants_reservations,restaurants_take_out,restaurants_counter_service,by_appointment_only,wheelchair_accessible,open_24_hours,dogs_allowed,caters,drive_thru,coat_check,good_for_dancing,restaurants_table_service,happy_hour,business_accepts_bitcoin,garage,street,validated,lot,valet,lunch,dinner,casual,city_state_le,name_le,max_user_likes,total_user_likes,avg_user_likes,max_business_likes,total_business_likes,avg_business_likes,max_ub_pair_likes,total_ub_pair_likes,avg_ub_pair_likes,rating
0,0,NCRGj36ihdwX-VUCYEX41g,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.13,317,0.378549,42.742889,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('NCRGj36ihdwX-VUCYEX41g', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,2.0
1,1,mySDXsBhY3u0yI-G2oVDkA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,2.73,239,0.200837,23.469196,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('mySDXsBhY3u0yI-G2oVDkA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,2.0
2,2,wLc6l8v7O9hXjrDi3Q7tyw,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.96,102,0.666667,11.978764,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('wLc6l8v7O9hXjrDi3Q7tyw', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,1.0,1.0,0.004808,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
3,3,hnhs-uzv_eoE6Pq8nXyufA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,3.84,203,0.394089,25.142518,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('hnhs-uzv_eoE6Pq8nXyufA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,0.0,0.0,0.000000,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
4,4,Gr-MqCunME2K_KmsAwjpTA,LNGBEEelQx4zbfWnlc66cw,4.0,35.965529,-115.168686,4.12,336,1.309524,37.922078,Henderson,NV,Studio B Buffet,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",2155,Henderson-NV,"('Gr-MqCunME2K_KmsAwjpTA', 'LNGBEEelQx4zbfWnlc...",2,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,357,117758,1.0,2.0,0.080000,2.0,11.0,0.012373,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142039,142039,NTuvVb-ZwQ_rFn6W9Krm7A,a9WV4-QcQfkUcOwUMFVhGw,4.5,36.076360,-115.081695,3.92,430,0.218605,54.973730,Henderson,NV,Complete Medical Consultants,"{'AcceptsInsurance': 'False', 'BusinessAccepts...",50,Henderson-NV,"('NTuvVb-ZwQ_rFn6W9Krm7A', 'a9WV4-QcQfkUcOwUMF...",0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,357,30789,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,5.0
142040,142040,qPganFH_fTXrP9r3kXf6YQ,aapwKXeDH_hsjthI13DI0w,4.0,45.572295,-73.754008,3.53,145,0.013793,28.968254,Laval,QC,The Keg Steakhouse + Bar - Laval,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",52,Laval-QC,"('qPganFH_fTXrP9r3kXf6YQ', 'aapwKXeDH_hsjthI13...",3,1,1,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,1,1,1,0,0,0,1,0,0,0,1,0,462,125874,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,4.0
142041,142041,zbRjeEkON8EXFk_VrcTAiA,QSMWY0fXT19xLfSVe9nXJg,1.5,36.078242,-115.093153,3.29,137,0.043796,13.588315,Las Vegas,NV,OnTrac,{'BusinessAcceptsCreditCards': 'True'},61,Las Vegas-NV,"('zbRjeEkON8EXFk_VrcTAiA', 'QSMWY0

In [7]:
final_test_df.keys()

Index(['Unnamed: 0', 'user_id', 'business_id', 'rounded_business_rating',
       'latitude', 'longitude', 'average_user_rating', 'num_user_reviews',
       'avg_user_compliments', 'reviews_per_year', 'city', 'state', 'name',
       'attributes', 'review_count', 'city_state', 'user_business_pair',
       'price_range', 'bike_parking', 'business_accepts_credit_cards',
       'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery',
       'restaurants_good_for_groups', 'restaurants_reservations',
       'restaurants_take_out', 'restaurants_counter_service',
       'by_appointment_only', 'wheelchair_accessible', 'open_24_hours',
       'dogs_allowed', 'caters', 'drive_thru', 'coat_check',
       'good_for_dancing', 'restaurants_table_service', 'happy_hour',
       'business_accepts_bitcoin', 'garage', 'street', 'validated', 'lot',
       'valet', 'lunch', 'dinner', 'casual', 'city_state_le', 'name_le',
       'max_user_likes', 'total_user_likes', 'avg_user_likes',
       'max_

In [8]:
len(final_test_df.keys())

59

In [9]:
X_train = train_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_good_for_groups', 'restaurants_reservations', 'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', \
            'open_24_hours', 'drive_thru', 'coat_check', 'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'lot', 'lunch', \
                'casual', 'city_state_le', 'name_le', 'total_user_likes', 'avg_user_likes', 'max_business_likes', 'total_business_likes', \
                    'avg_business_likes', 'avg_ub_pair_likes']]
y_train = train_df['rating']

xgb = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, \
    gamma=0, gpu_id=1, interaction_constraints='', learning_rate=0.300000012, max_delta_step=0, max_depth=6, min_child_weight=1, \
        n_jobs=8, num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1, \
            tree_method='exact', validate_parameters=1, verbosity=None).fit(X_train, y_train)

y_pred_train = list(xgb.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

X_test = final_test_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_good_for_groups', 'restaurants_reservations', 'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', \
            'open_24_hours', 'drive_thru', 'coat_check', 'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'lot', 'lunch', \
                'casual', 'city_state_le', 'name_le', 'total_user_likes', 'avg_user_likes', 'max_business_likes', 'total_business_likes', \
                    'avg_business_likes', 'avg_ub_pair_likes']]
y_test = final_test_df['rating']
y_pred_test = list(xgb.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9627557869397649; Test RMSE: 0.9802865631165782


In [10]:
X_train = train_df[['rounded_business_rating',
       'latitude', 'longitude', 'average_user_rating', 'num_user_reviews',
       'avg_user_compliments', 'reviews_per_year',
       'review_count',
       'price_range', 'bike_parking', 'business_accepts_credit_cards',
       'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery',
       'restaurants_good_for_groups', 'restaurants_reservations',
       'restaurants_take_out', 'restaurants_counter_service',
       'by_appointment_only', 'wheelchair_accessible', 'open_24_hours',
       'dogs_allowed', 'caters', 'drive_thru', 'coat_check',
       'good_for_dancing', 'restaurants_table_service', 'happy_hour',
       'business_accepts_bitcoin', 'garage', 'street', 'validated', 'lot',
       'valet', 'lunch', 'dinner', 'casual', 'city_state_le', 'name_le',
       'max_user_likes', 'total_user_likes', 'avg_user_likes',
       'max_business_likes', 'total_business_likes', 'avg_business_likes',
       'max_ub_pair_likes', 'total_ub_pair_likes', 'avg_ub_pair_likes']]
y_train = train_df['rating']

X_test = final_test_df[['rounded_business_rating',
       'latitude', 'longitude', 'average_user_rating', 'num_user_reviews',
       'avg_user_compliments', 'reviews_per_year',
       'review_count',
       'price_range', 'bike_parking', 'business_accepts_credit_cards',
       'good_for_kids', 'has_TV', 'outdoor_seating', 'restaurants_delivery',
       'restaurants_good_for_groups', 'restaurants_reservations',
       'restaurants_take_out', 'restaurants_counter_service',
       'by_appointment_only', 'wheelchair_accessible', 'open_24_hours',
       'dogs_allowed', 'caters', 'drive_thru', 'coat_check',
       'good_for_dancing', 'restaurants_table_service', 'happy_hour',
       'business_accepts_bitcoin', 'garage', 'street', 'validated', 'lot',
       'valet', 'lunch', 'dinner', 'casual', 'city_state_le', 'name_le',
       'max_user_likes', 'total_user_likes', 'avg_user_likes',
       'max_business_likes', 'total_business_likes', 'avg_business_likes',
       'max_ub_pair_likes', 'total_ub_pair_likes', 'avg_ub_pair_likes']]
y_test = final_test_df['rating']

In [11]:
for feature in X_train.keys():
    print(train_df[[feature, "rating"]].corr())
    print()

                         rounded_business_rating    rating
rounded_business_rating                 1.000000  0.385519
rating                                  0.385519  1.000000

          latitude    rating
latitude  1.000000 -0.036627
rating   -0.036627  1.000000

           longitude    rating
longitude   1.000000 -0.022774
rating     -0.022774  1.000000

                     average_user_rating   rating
average_user_rating              1.00000  0.32647
rating                           0.32647  1.00000

                  num_user_reviews    rating
num_user_reviews          1.000000 -0.022483
rating                   -0.022483  1.000000

                      avg_user_compliments    rating
avg_user_compliments              1.000000  0.014383
rating                            0.014383  1.000000

                  reviews_per_year    rating
reviews_per_year          1.000000 -0.005768
rating                   -0.005768  1.000000

              review_count    rating
review_count      1.

In [17]:
n = len(list(X_train.keys()))-1
pbar = tqdm(total=n)
model_performances_with_features = {}
while n >= 20:
    test_xgb = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, \
    gamma=0, gpu_id=1, interaction_constraints='', learning_rate=0.300000012, max_delta_step=0, max_depth=6, min_child_weight=1, \
        n_jobs=8, num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1, \
            tree_method='exact', validate_parameters=1, verbosity=None)
    rfe = RFE(test_xgb, n_features_to_select=n)

    rfe.fit(X_train, y_train)
    features_supp = rfe.support_
    features_out = [list(X_train.keys())[i] for i, supp in enumerate(features_supp) if supp == True]

    test_xgb.fit(X_train[features_out], y_train)
    y_pred_train_rfe = list(test_xgb.predict(X_train[features_out]))
    train_rmse_rfe = mean_squared_error(list(y_train), y_pred_train_rfe, squared=False)

    y_pred_test_rfe = list(test_xgb.predict(X_test[features_out]))
    test_rmse_rfe = mean_squared_error(list(y_test), y_pred_test_rfe, squared=False)
    print(f'##### {n} #####\nTrain RMSE: {train_rmse_rfe}; Test RMSE: {test_rmse_rfe}\n')
    n -= 1
    model_performances_with_features[(train_rmse_rfe, test_rmse_rfe)] = features_out
    pbar.update(1)

pbar.close()
best_model = min(model_performances_with_features.items(), key=lambda x: x[0][1])
print(f'\nBest Model:\nTrain RMSE: {best_model[0][0]}; Test RMSE: {best_model[0][1]}\nBest Features: {best_model[1]}')

  0%|          | 0/48 [00:05<?, ?it/s]


##### 48 #####
Train RMSE: 0.9631763199027813; Test RMSE: 0.9802364771597247



##### 47 #####
Train RMSE: 0.9629896644667604; Test RMSE: 0.9800836754112154



##### 46 #####
Train RMSE: 0.9628310124715248; Test RMSE: 0.9805718375270283



##### 45 #####
Train RMSE: 0.9631631552543403; Test RMSE: 0.9800139521960902



...

##### 36 #####
Train RMSE: 0.9627640271815966; Test RMSE: 0.980334512008377



##### 35 #####
Train RMSE: 0.9628918828554006; Test RMSE: 0.9802457769905818



##### 34 #####
Train RMSE: 0.9619329783843893; Test RMSE: 0.9801145726597089



##### 33 #####
Train RMSE: 0.9634382906180425; Test RMSE: 0.980236025472235



KeyboardInterrupt: 

In [18]:
best_model = min(model_performances_with_features.items(), key=lambda x: x[0][1])
print(f'\nBest Model:\nTrain RMSE: {best_model[0][0]}; Test RMSE: {best_model[0][1]}\nBest Features: {best_model[1]}')


Best Model:
Train RMSE: 0.9630116041852984; Test RMSE: 0.9798764280445167
Best Features: ['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', 'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', 'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', 'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', 'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', 'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']


In [20]:
X_train = train_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', \
            'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', \
                'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                    'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']]
y_train = train_df['rating']

xgb = XGBRegressor(objective='reg:linear', base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, \
    gamma=0, gpu_id=1, interaction_constraints='', learning_rate=0.300000012, max_delta_step=0, max_depth=6, min_child_weight=1, \
        n_jobs=8, num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1, \
            tree_method='exact', validate_parameters=1, verbosity=None).fit(X_train, y_train)

y_pred_train = list(xgb.predict(X_train))
train_rmse = mean_squared_error(list(y_train), y_pred_train, squared=False)

X_test = final_test_df[['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', \
            'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', \
                'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                    'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']]
y_test = final_test_df['rating']
y_pred_test = list(xgb.predict(X_test))
test_rmse = mean_squared_error(list(y_test), y_pred_test, squared=False)

print(f'Train RMSE: {train_rmse}; Test RMSE: {test_rmse}')

Train RMSE: 0.9630116041852984; Test RMSE: 0.9798764280445167


In [ ]:
['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', 'avg_user_compliments', \
    'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'outdoor_seating', 'restaurants_delivery', \
        'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'dogs_allowed', 'caters', 'drive_thru', 'coat_check', \
            'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'street', 'lot', 'valet', 'lunch', 'dinner', 'casual', \
                'city_state_le', 'name_le', 'max_user_likes', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                    'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']

In [ ]:
['rounded_business_rating', 'latitude', 'longitude', 'average_user_rating', 'num_user_reviews', \
        'avg_user_compliments', 'reviews_per_year', 'review_count', 'price_range', 'bike_parking', 'good_for_kids', 'has_TV', \
            'outdoor_seating', 'restaurants_delivery', 'restaurants_good_for_groups', 'restaurants_reservations', \
                'restaurants_take_out', 'by_appointment_only', 'wheelchair_accessible', 'open_24_hours', 'drive_thru', \
                    'coat_check', 'restaurants_table_service', 'business_accepts_bitcoin', 'garage', 'lot', 'lunch', 'casual', \
                        'city_state_le', 'name_le', 'total_user_likes', 'avg_user_likes', 'max_business_likes', \
                            'total_business_likes', 'avg_business_likes', 'avg_ub_pair_likes']

In [ ]:
{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'monotone_constraints': '()',
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}